# Interactive Syd Viewer (with Seaborn)
This example shows you how to use Syd to create interactive plots that use Seaborn. 

### Main concepts: 
1. Syd is based on matplotlib, so the backend of matplotlib determines the interactivity of your viewer! In general this means you should use %matplotlib widget in your notebook to turn on the standard matplotlib interactivity widgets. 

2. Seaborn is also built on top of matplotlib, so it has the same interactivity functionality -- if you are expecting to have a seaborn-style interactive plot and aren't getting it, turn on %matplotlib widget in your notebook. 

3. Just so you know - any plotting package built on top of matplotlib is useable in Syd! The only rule is that the plot function needs to return a matplotlib figure object. 

4. Interactivity with the plot itself doesn't really work in the web browser view. This is because it isn't implemented - not because it isn't possible. So, if you feel like you really need that, please raise an issue on the github to let us know we should work on it!

In [1]:
try:
    import google.colab
    # We're in Colab
    !pip install git+https://github.com/landoskape/syd.git

except ImportError:
    pass

In [ ]:
%matplotlib widget

import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from syd import Viewer
sns.set_theme(style="ticks")

rs = np.random.RandomState(11)

# Viewer definition
class SeabornViewer(Viewer):
    def __init__(self):
        self.add_float("gamma", value=1.0, min=0.1, max=5.0)
        self.add_float("slope", value=0.0, min=-5.0, max=5.0)
        self.add_selection("color", value="red", options=["red", "blue", "green", "black"])
        self.add_selection("plot_type", value="jointplot", options=["jointplot", "scatterplot", "histplot", "kdeplot"])
        self.add_button("save", label="Save Figure", callback=self.save_figure, replot=False)

    def save_figure(self, state):
        self.figure.savefig("seaborn_figure.png")

    def plot(self, state):
        x = rs.gamma(state["gamma"], size=1000)
        y = state["slope"] * x + rs.normal(size=1000)

        if state["plot_type"] == "jointplot":
            grid = sns.jointplot(x=x, y=y, kind="hex", color=state["color"])
            fig = grid._figure

        else:
            fig, ax = plt.subplots(figsize=(6, 4), layout="constrained")

            if state["plot_type"] == "scatterplot":
                sns.scatterplot(x=x, y=y, color=state["color"], ax=ax)

            elif state["plot_type"] == "histplot":
                sns.histplot(x=x, y=y, color=state["color"], ax=ax)

            elif state["plot_type"] == "kdeplot":
                sns.kdeplot(x=x, y=y, color=state["color"], ax=ax)

            else:
                raise ValueError(f"Invalid plot type: {state['plot_type']}")
        
        return fig

viewer = SeabornViewer()
viewer.show()